### Pytorch 加速
DEAP除了可以使用Numpy进行CPU加速，还可以基于PyTorch进行GPU加速。值得一提的是，在评估时，数据需要以Tensor的格式加载，传统的Numpy格式不能被GPU处理。这种加速方式可以显著提高计算速度，特别是对于大规模的模型评估。

In [1]:
import time

import torch
from deap import base, creator, tools, gp

# 确保你的GPU可用，如果不可用，则在CPU上运行
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)

   # 使用torch创建一个张量
    xs = torch.linspace(-10, 10, 100, device=device)
    
    # 计算均方误差（MSE）
    ys = torch.pow(xs, 2)
    squared_errors = torch.pow(func(xs) - ys, 2)
    mse = torch.mean(squared_errors).item()  # 将结果转换回Python float
    
    return (mse,)

# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

对应的算子一定要替换成PyTorch函数。

In [2]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(torch.add, 2)
pset.addPrimitive(torch.sub, 2)
pset.addPrimitive(torch.mul, 2)
pset.addPrimitive(torch.neg, 1)
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

现在，我们可以开始基于PyTorch进行GPU加速了。

In [3]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
start=time.time()
population = toolbox.population(n=50)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
end=time.time()
print('time:',end-start)
print(str(hof[0]))

   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max    	min	nevals	std    	avg 	gen	max	min	nevals	std    
0  	50    	2489.32	0  	18732.1	0  	50    	2898.35	3.78	0  	7  	2  	50    	1.68867
1  	29    	7743.9 	1  	153712 	0  	29    	29842  	4.08	1  	9  	2  	29    	1.62284
2  	31    	8699.28	2  	170363 	0  	31    	31485.2	4.36	2  	10 	3  	31    	1.65843
3  	29    	954266 	3  	4.75033e+07	0  	29    	6.64989e+06	4.16	3  	9  	2  	29    	1.6536 
4  	41    	14754.3	4  	153712     	0  	41    	41173.8    	4.5 	4  	9  	3  	41    	1.80278
5  	34    	257120 	5  	1.23323e+07	0  	34    	1.72542e+06	4.16	5  	10 	3  	34    	1.86933
6  	29    	7108.62	6  	153712     	0  	29    	30012.1    	3.76	6  	9  	3  	29    	1.33507
7  	33    	238886 	7  	1.17257e+07	0  	33    	1.64112e+06	3.62	7  	9  	2  	33    	1.35484
8  	28  